# Final Project Baseline
Yang Wei Neo, Emily Rapport, Hilary Yamtich

## Load Libraries and Data

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import csv
from rfpimp import *
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

# note: this notebook requires pandas 0.21.0 or newer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime as dt
import re as re
import pickle as pk
from sklearn.pipeline import Pipeline

# For producing decision tree diagrams.
from IPython.core.display import Image, display
from sklearn.externals.six import StringIO

from dateutil import parser
import datetime

In [ ]:
# Basic EDA

# The training data needs to be saved in the same location.
original_data = pd.read_csv('train.csv')
data = original_data.copy()

# We also read in kaggle testing data so that we can run same feature eng steps
original_test_data = pd.read_csv('test.csv')
test_data = original_test_data.copy()

# Background to the problem

### The Dataset and Framing the Problem

The goal of this problem is to use the Ames Housing Dataset to create a model to predict the sale prices of homes randomly selected from within the original dataset. 

The Ames Housing Dataset was compiled by Dean De Cock in 2011. It is based on all the housing sales that occured in Ames, Iowa during the period from 2006 to 2010. It was created as an alternative to the Boston Housing Dataset and it includes more feature variables (79) that are related to housing price. The original dataset included 2,390 observations. The goal of the creator was to include a wider set of explanatory variables than the original Boston Housing Dataset to allow for more advanced regression techniques to be applied.

The problem itself is based on Kaggle competition to use advanced regression techniques to create a model to predict the sale price of a home. The test data is randomly selected from within the original Ames Housing Dataset. The training dataset used in the Kaggle competition includes 1,460 observations. The goal is to accurately predict the sale prices of the homes in the test set.

### Distribution of Sale Price Variable

First we look at the distribution of the sale price variable using a histogram.

In [ ]:
data.hist("SalePrice", bins = 50)
plt.xlabel("Sale Prices")

In [ ]:
data["LogSalePrice"] = np.log(data["SalePrice"])
data.hist("LogSalePrice", bins = 50)

This produces a more normally distributed set of data. This makes our highly skewed distribution less skewed and allows our sale price data to meet the assumptions of inferential statistics and linear regression.

## Exploratory Data Analysis

## Feature Engineering

### Overview

We have two broad approaches to feature engineering - 'top down' and 'bottom up'. 

In the 'top down' case, we start with all the variables and work to eliminate / simplify variables as much as possible. We remove features with little useful variation (mostly missing, or heavily imbalanced) and features that are highly correlated for obvious reasons (e.g. total square feet and other component square feet). These should help to reduce the likelihood of overfitting and help improve interpretability. 

In the 'bottom up' case, we use various algorithmic feature selection techniques (e.g. random forest feature importance) and choose the top 10. This helps us construct the most parsimonious feature set to compare our more complete feature set against. 

In both cases, we also make several variable transformations - imputing missing features, or one-hot-encoding categorical features, for instance. They are listed below. 

### Remove features with very little variation and those that are missing for most observations

Based on this heatmap of missing variables we see that Alley, PoolQC, Fence and MiscFeature are missing for most values in the dataset. This makes sense, as these are more esoteric features of the house. In the same vein, there is also very little variation in Street, Utilities, LowQualFinSF and MiscVal. For example, most of these houses have all the utilities except for one. While this may ultimately be an important predictor, we believe that this is more likely to overfit and not generalize (after all this observation can only be in validation or train) for any given fold.

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(original_data.isnull(), cbar=False)

def remove_low_var_and_missin_features(inp_data):
    inp_data = inp_data.drop(columns=['Alley', 'PoolQC', 'Fence', 'MiscFeature'])

    # Drop variables with very imblanced categories or mostly missing
    inp_data = inp_data.drop(columns = ['Street', 'Utilities', 'LowQualFinSF', 'MiscVal'])
    
    return inp_data

data = remove_low_var_and_missin_features(data)
test_data = remove_low_var_and_missin_features(test_data)

### Remove features that are redundant given other variables

We then remove aggregated variables that are very close to the sum of its component parts. For instance, TotalBsmtSF is the approximately sum of the BsmtFinSF1, BSmtUnfSF and BsmtFinSF2. Likewise with GrLivArea. There are some minor discrepancies, but I assume that those are immaterial for our analysis and not large/frequent enough to matter. 

GarageArea / GarageCars is an interesting case, as these two variables capture a similar concept. Indeed, the two variables are highly correlated (see chart below). We keep both in for now, but may decide to remove the 'less informative' one later (e.g. GarageCars). 

In [ ]:
def remove_redundant_features(inp_data):
    ##################
    ### BASEMENTSF ###
    # Drop the aggregate BsmtSF variable
    test = pd.DataFrame()
    test['DiffBsmtSF'] = original_data["TotalBsmtSF"] - original_data['BsmtFinSF1'] - original_data['BsmtUnfSF'] - original_data['BsmtFinSF2']
    test['DiffBsmtSF'].value_counts()

    # Remove variable
    inp_data = inp_data.drop(columns=['TotalBsmtSF'])

    ##################
    ### GRLIVAREA ###
    # Is GrLivArea simply the sum of 1st and 2nd?
    # Most of hte time it is - there are 5 observations that are not, but doesn't seem like a goood reason to keep
    test = pd.DataFrame()
    test['GrLivArea'] = original_data["GrLivArea"]
    test['1stAnd2nd'] = original_data["1stFlrSF"] + original_data["2ndFlrSF"]
    test['Diff'] = test['GrLivArea'] - test['1stAnd2nd']
    test['Diff'].value_counts()

    ###################
    ### GARAGECARS ###
    inp_data = inp_data.drop(columns = ['GrLivArea'])
    original_data['GarageCars'].value_counts()
    plt.scatter(original_data['GarageCars'],original_data['GarageArea'])
    #data = data.drop(columns=['GarageCars']) # KEEP FOR NOW, but garage area may be better than garage cars

    ###################
    ### DATETIME    ###
    # Change date to ordinal representation:
    inp_data['YrMoSold'] = inp_data['YrSold'].astype(int).astype(str) + "/" + inp_data['MoSold'].astype(int).astype(str) + "/1"
    inp_data['YrMoSold'] = inp_data['YrMoSold'].apply(lambda x: dt.strptime(x, "%Y/%m/%d").date().toordinal())
    return inp_data
    
data = remove_redundant_features(data)
test_data = remove_redundant_features(test_data)

In [ ]:
## save copy of lot frontage - original 
data_LotFrontage_original = data['LotFrontage'].copy()

def impute_missing_vals(inp_data):
    # Impute all numeric missings (all dummy missings are coded 0)
    imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
    data_numeric = imp.fit_transform(inp_data.select_dtypes(exclude='object'))
    data_numeric = pd.DataFrame(data_numeric, columns = inp_data.select_dtypes(exclude='object').columns)
    inp_data = pd.concat([inp_data.select_dtypes(include='object'), data_numeric], axis = 1)
    return inp_data

data = impute_missing_vals(data)
test_data = impute_missing_vals(test_data)

### Combine sparse categories in some features

Condition and Exterior features have many sparse levels - we group levels that have very few observations to obtain a bigger category. 

In [ ]:
def combine_thin_categories(inp_data):
    # Combine categories - very thin categories
    # Condition1
    inp_data['Condition1'] = np.where(inp_data['Condition1']=='Norm', 'Norm', 'Not_Norm')
    inp_data['Condition2'] = np.where(inp_data['Condition2']=='Norm', 'Norm', 'Not_Norm')

    # Exterior1st
    inp_data['Exterior1st'] = np.where(inp_data['Exterior1st'].isin(['BrkComm', 'Stone', 'AsphShn', 'ImStucc', "CBlock", "WdShing", "Stucco", "AsbShng"]), 'Other', inp_data['Exterior1st'])
    inp_data['Exterior2nd'] = np.where(inp_data['Exterior2nd'].isin(['Brk Cmn', 'Stone', 'AsphShn', 'Other', "CBlock", "WdShing", "Stucco", "AsbShng"]), 'Other', inp_data['Exterior1st'])

    return inp_data

data = combine_thin_categories(data)
test_data = combine_thin_categories(test_data)

### One-Hot-Encoding for Categoricals

We use one-hot encoding to encode categorical features and combine the condition and exterior variables into one variable each. 

In [ ]:
def one_hot_encode(inp_data):
    ##########################
    # Binarize all categorical (one-hot)
    inp_data = pd.get_dummies(inp_data)

    # Simplify the Condition Feature
    inp_data['Condition_Norm'] = inp_data['Condition1_Norm'] + inp_data['Condition2_Norm']
    inp_data['Condition_Not_Norm'] = inp_data['Condition1_Not_Norm'] + inp_data['Condition2_Not_Norm']
    inp_data['Condition_Norm'] = np.where(inp_data['Condition_Norm']==2, 1, inp_data['Condition_Norm'])
    inp_data['Condition_Not_Norm'] = np.where(inp_data['Condition_Not_Norm']==2, 1, inp_data['Condition_Not_Norm'])

    # Simplify the Exterior Feature
    inp_data['Exterior_VinylSd'] = inp_data['Exterior1st_VinylSd'] + inp_data['Exterior2nd_VinylSd']
    inp_data['Exterior_HdBoard'] = inp_data['Exterior1st_HdBoard'] + inp_data['Exterior2nd_HdBoard']
    inp_data['Exterior_MetalSd'] = inp_data['Exterior1st_MetalSd'] + inp_data['Exterior2nd_MetalSd']
    inp_data['Exterior_Wd Sdng'] = inp_data['Exterior1st_Wd Sdng'] + inp_data['Exterior2nd_Wd Sdng']
    inp_data['Exterior_Plywood'] = inp_data['Exterior1st_Plywood'] + inp_data['Exterior2nd_Plywood']
    inp_data['Exterior_Other']   = inp_data['Exterior1st_Other']   + inp_data['Exterior2nd_Other']
    inp_data['Exterior_CemntBd'] = inp_data['Exterior1st_CemntBd'] + inp_data['Exterior2nd_CemntBd']
    inp_data['Exterior_BrkFace'] = inp_data['Exterior1st_BrkFace'] + inp_data['Exterior2nd_BrkFace']

    inp_data['Exterior_VinylSd'] = np.where(inp_data['Exterior_VinylSd'] == 2, 1, inp_data['Exterior_VinylSd'])
    inp_data['Exterior_HdBoard'] = np.where(inp_data['Exterior_HdBoard'] == 2, 1, inp_data['Exterior_HdBoard'])
    inp_data['Exterior_MetalSd'] = np.where(inp_data['Exterior_MetalSd'] == 2, 1, inp_data['Exterior_MetalSd'])
    inp_data['Exterior_Wd Sdng'] = np.where(inp_data['Exterior_Wd Sdng'] == 2, 1, inp_data['Exterior_Wd Sdng'])
    inp_data['Exterior_Plywood'] = np.where(inp_data['Exterior_Plywood'] == 2, 1, inp_data['Exterior_Plywood'])
    inp_data['Exterior_Other'] = np.where(inp_data['Exterior_Other'] == 2, 1, inp_data['Exterior_Other'])
    inp_data['Exterior_CemntBd'] = np.where(inp_data['Exterior_CemntBd'] == 2, 1, inp_data['Exterior_CemntBd'])
    inp_data['Exterior_BrkFace'] = np.where(inp_data['Exterior_BrkFace'] == 2, 1, inp_data['Exterior_BrkFace'])

    cols_to_drop = list(filter(lambda x: re.search(r'Condition1|Condition2|Exterior1|Exterior2', x), inp_data.columns))
    inp_data = inp_data.drop(columns=cols_to_drop)
    
    return inp_data

data = one_hot_encode(data)
test_data = one_hot_encode(test_data)

In [ ]:
test_data.shape

### Correct Test Data to Match Train Data's One-Hot Encoding

When we one-hot encode, the number of new columns we create depends on the number of classes contained in each column. For instance, if our data has 10 possible values for MSSubClass, then we'll end up with 10 new columns, one for each of those values. However, when we do the same operation on our test data, we end up with a different set of columns, since the data in the two sets is different. We could end up with features that are only in our train data, or features that are only in our test data. 

In [ ]:
# detect features that are only in our train data
for column in data.columns:
    if column not in test_data.columns:
        print(column)

In [ ]:
# detect features that are only in our test data 
for column in test_data.columns:
    if column not in data.columns:
        print(column)

If our model trains on features that are present in the train data, it will need to have the same columns available in the test data. Since these columns represent particular values of a categorical variable that just weren't present in our test data, we can simply add the columns to our test data with a 0 for each row. 

In [ ]:
for col in data.columns:
    if col not in test_data.columns:
        test_data[col] = 0
test_data = test_data[[col for col in data.columns if col not in ['LogSalePrice', 'SalePrice']]]

## confirm that the only columns that differ between the two sets are now the outcome variables,
## which our test data should not have

print([column for column in data.columns if column not in test_data.columns])

A critical final step: re-order the columns in the test data frame so that they're in the same order as the train data frame. Otherwise, scikit-learn will appear to be predicting correctly, but will actually be looking at the wrong variables and make bad predictions (this actually happened to us for a few days). 

In [ ]:
train_col_order = [col for col in data.columns if col not in ['SalePrice', 'LogSalePrice']]
test_data = test_data[train_col_order]

## confirm that columns are now in same order
## we should see an array of "trues" 
train_col_order == test_data.columns

### Convert features to the right type

In [ ]:
def convert_feature_types(inp_data, original):
    inp_data['OverallQual'] = original['OverallQual'].astype('category')
    inp_data['OverallCond'] = original['OverallCond'].astype('category', ordered=True)
    inp_data['MSSubClass'] = original['MSSubClass'].astype('category')
    return inp_data

data = convert_feature_types(data, original_data)
test_data = convert_feature_types(test_data, original_test_data)

### Impute features

- There are several variables that are missing substantial amount of data; however, one of them is especially critical for predicting home prices - the Lot Frontage. We use a kNN to impute the missing lot frontages - a cursory look at the distribution shows that the values follow a tighter, but normal-ish distribution. This is similar to the distribution of the nonmissing data. 

- Not shown here is a series of tests we ran to compare the accuracy when using the median imputation vs. the more precise kNN imputation. The results are very close (difference is to the fourth decimal place). In linear regression, the kNN imputation does marginally better regardless of the regularization used. In Random Forest - the model with kNN imputed values actually does worse for iterations with simpler trees and better for iterations with more complex trees. We suspect that this might be because more complex trees may be more likely to overfit imputed values.

- Regardless, we think the evidence points marginally in favor of using the imputation. 

In [ ]:
# TODO: YW - can you turn this into a function and handle train/test however you think is appropriate?
# Impute one or two key variables that are missing a substantial amount of data
# Normalize data before running 
scaler = StandardScaler(copy=True)
data_scaled = scaler.fit_transform(data.values)
data_scaled = pd.DataFrame(data_scaled, index = data.index, columns = data.columns)

# For example, Lot Frontage
knn = KNeighborsRegressor(n_neighbors=5, weights = 'distance')

# Isolate training data and missing data
notmissing_flag = data_LotFrontage_original.notnull()
missing_flag = data_LotFrontage_original.isnull()

# Define test data
lotfrontage_data = data_scaled[notmissing_flag].drop(columns='LotFrontage')
lotfrontage_labels = data_LotFrontage_original[notmissing_flag]

# test missing (prediction data)
lotfrontage_testdata = data_scaled.drop(columns='LotFrontage')

# Use KNN to predict missing data
knn.fit(lotfrontage_data, lotfrontage_labels)
lotpredictions = knn.predict(lotfrontage_testdata).astype(int)

# Plug it back into training data
data_LotFrontage_imputed = data_LotFrontage_original.copy()
data_LotFrontage_imputed[missing_flag] = lotpredictions
data_LotF_df = pd.DataFrame(np.column_stack((data_LotFrontage_imputed, missing_flag)), columns = ['LotFrontage_Imputed', 'Missing'])

# Plot the two histograms:
dataforplot = data_LotF_df.groupby('Missing')['LotFrontage_Imputed']
dataforplot.plot(kind='hist', bins = 30, density = True, histtype = 'step')

# Plug back into the data
data_LotFrontage_imputed_df = pd.DataFrame({'LotFrontage_Imputed': data_LotFrontage_imputed})
data = pd.concat([data.drop(columns='LotFrontage'), data_LotFrontage_imputed_df], axis = 1)

## todo: this is just a hack for now so that they'll match - fix this after we decide how to handle all this
test_data['LotFrontage_Imputed'] = test_data['LotFrontage']

### Add new features

In [ ]:
def add_new_features(inp_data):
    inp_data['Basement'] = np.where(inp_data['BsmtFinSF1'] + inp_data['BsmtFinSF2'] + inp_data['BsmtUnfSF'] > 1, 1, 0)
    inp_data['WoodDeck'] = np.where(inp_data['WoodDeckSF'] > 0, 1, 0)
    return inp_data

data = add_new_features(data)
train_data = add_new_features(test_data)

### PCA

First we tried PCA without scaling and found that the first Principal Component captured 99% of the variation! This seemed too good to be true - indeed it was as our underlying data was not normalized or standardized. So most of the variation came from variables that were very large in units (e.g. LotSize). We validated this and saw that the units of the PCA itself were uncannily similar to LotSize; and that the principal components were also highly correlated with the LotSize variable.

After normalizing the underlying data we found that the PCA made more sense: even 50 components only explained 75% of the variance. 

In [ ]:
# todo YW: can you handle train/test however you think is appropriate here?
# Initialize parameters 
num_components = 50
data_for_pca = data.drop(columns=['YrMoSold', 'SalePrice', 'Id', 'LogSalePrice'])
pca_names = ['pca_' + str(i) for i in range(num_components)]

# First scale
scaler = StandardScaler(copy=True)
data_scaled = scaler.fit_transform(data_for_pca.values)
data_scaled = pd.DataFrame(data_scaled, index = data_for_pca.index, columns = data_for_pca.columns)

# Fit PCAe
pca = PCA(n_components=num_components)
data_pca = pca.fit_transform(data_scaled)
data_pca = pd.DataFrame(data_pca, columns=pca_names)

# Show Results
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Num Components')
plt.ylabel('Incremental Explained Variance')

In [ ]:
# Cbind to dataframe
data = pd.concat([data.reset_index(drop=True), data_pca], axis = 1)

### Feature Interactions

- First we investigate the pairwise correlation between all variables and identify any set variables that are highly correlated. This led us to discover the redundant sets of variables discussed above.
- Further, we examine what variables seem to drive most of the variation in the dataset (by looking at those that are highly correlated with the top 5 principal components). Based on the PCA we might surmise that most of the variation in homes is associated with the number/types of rooms and the size of home; less so the meta-attributes (e.g. year sold). And, as expected, the principal components are uncorrelated with each other, by construction. 
    - PCA_0 seems to be capturing most of the variation that is linearly correlated with SalePrice (even though saleprice and its derivatives were not used in the generation of the PCA)
    - PCA_1 seems to be capturing variation associated with the discrete features in the home (e.g. number of baths, rooms, kitchen etc.)
    - PCA_2 seems to be capturing variation associated with the size of the home (SF, Frontage, LotArea, etc.)
    
- In testing (not shown here), PCA with the top 50 components does not improve accuracy (it is actually worse!) It is betwen 1-2 points worse for linear regression and for random forest on various combinations. We decide to exclude the use of the principal components for modeling. 

In [ ]:
data_for_visual = original_data.select_dtypes(exclude='object')
data_for_visual = pd.concat((data_for_visual, data_pca.iloc[:,0:5]), axis = 1)

# Correlation matrix
plt.figure(figsize=(20,10))
corr = data_for_visual.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .7})

### Algorithmic Feature Selection method
- We use this to sense check the range of important features. We find several interesting observations:
    - Principal components are not necessarily the most predictive of the target variable - other variables, such as size-related variables still rank higher than several of the top 5 principal components - See below. This makes sense because PCA only tries to create components that maximize the variance of the feature vectors and does not consider the target variable. 

In [ ]:
# Remove target variables
remove_variables = ['SalePrice', 'LogSalePrice']

# Use random forest to remove feature vectors
rf = RandomForestRegressor()
rf.fit(data.drop(columns=remove_variables), data['SalePrice'])

# Set up data frame to show feature importance including PCA variables
feature_importances = pd.DataFrame(rf.feature_importances_.round(4),
                                   index = data.drop(columns=remove_variables).columns, 
                                   columns=['importance']).sort_values('importance', ascending=False)

print(feature_importances[0:10])

# Set up data frame to show feature importance excluding PCA variables
remove_variables_pca = ['SalePrice', 'LogSalePrice', *pca_names]

rf.fit(data.drop(columns=remove_variables_pca), data['SalePrice'])
feature_importances_wopca = pd.DataFrame(rf.feature_importances_.round(4),
                                   index = data.drop(columns=remove_variables_pca).columns, 
                                   columns=['importance']).sort_values('importance', ascending=False)

print(feature_importances_wopca[0:10])

### Export the Data

In [ ]:
# Final data to CSV file
data.to_csv('clean_data.csv')

# Final data to pickle file
outfile = open('clean_data_pickle','wb')
pk.dump(data, outfile)
outfile.close()

# final test data to csv and pickle files
test_data.to_csv('clean_test_data.csv')
with open('clean_test_data.pkl', 'wb') as outfile:
    pk.dump(test_data, outfile)

#saving this off for use in next notebook
feature_importances_wopca.to_csv('feature_importance.csv')